# Create TOBS 2025 DataFrame

This notebook creates a wide-format dataframe for TOBS (Temperature at observation time) data for 2025, where:
- Each row represents a station
- Columns include station ID, year, and 365 columns for each day of the year
- Values are TOBS measurements (NaN if not defined)


In [ ]:
# Import libraries
import s3fs
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

print('✓ Libraries imported successfully!')


In [ ]:
# Connect to S3
print('Connecting to S3...')
s3 = s3fs.S3FileSystem(anon=True)
bucket_name = 'noaa-ghcn-pds'
print(f'✓ Connected to S3 bucket: {bucket_name}')


In [ ]:
# Read TOBS data for 2025
tobs_path = f's3://{bucket_name}/parquet/by_year/YEAR=2025/ELEMENT=TOBS/'
print(f'Reading TOBS data for 2025 from: {tobs_path}')

# Read all parquet files for TOBS 2025
df_long = pd.read_parquet(tobs_path, storage_options={'anon': True})

print(f'\nData loaded:')
print(f'  - Shape: {df_long.shape[0]:,} rows × {df_long.shape[1]} columns')
print(f'  - Unique stations: {df_long["ID"].nunique():,}')
print(f'  - Date range: {df_long["DATE"].min()} to {df_long["DATE"].max()}')
print(f'\nFirst few rows:')
print(df_long.head())


In [ ]:
# Convert DATE column to datetime and extract day of year
df_long['date_dt'] = pd.to_datetime(df_long['DATE'], format='%Y%m%d')
df_long['day_of_year'] = df_long['date_dt'].dt.dayofyear
df_long['year'] = df_long['date_dt'].dt.year

# Check if 2025 is a leap year (it's not)
days_in_2025 = 365
print(f'Days in 2025: {days_in_2025}')
print(f'Day of year range: {df_long["day_of_year"].min()} to {df_long["day_of_year"].max()}')


In [ ]:
# Keep only necessary columns and rename for clarity
df_pivot = df_long[['ID', 'year', 'day_of_year', 'DATA_VALUE']].copy()

# Pivot the data: stations as rows, days as columns
print('Pivoting data to wide format...')
df_wide = df_pivot.pivot_table(
    index=['ID', 'year'],
    columns='day_of_year',
    values='DATA_VALUE',
    aggfunc='first'  # Use first value if multiple observations per day
)

# Reset index to make ID and year regular columns
df_wide = df_wide.reset_index()

print(f'\nWide format dataframe created:')
print(f'  - Shape: {df_wide.shape[0]:,} rows × {df_wide.shape[1]} columns')
print(f'  - Columns: station_id, year, day_1, day_2, ..., day_{days_in_2025}')
print(f'\nFirst few rows and columns:')
print(df_wide.iloc[:5, :10])


In [ ]:
# Rename columns to be more descriptive
# Ensure all days 1-365 exist (add missing columns with NaN)
for day in range(1, days_in_2025 + 1):
    if day not in df_wide.columns:
        df_wide[day] = np.nan

# Sort columns: ID, year, then days 1-365 in order
day_columns = sorted([col for col in df_wide.columns if isinstance(col, int)])
df_wide = df_wide[['ID', 'year'] + day_columns]

# Rename columns
df_wide.columns = ['station_id', 'year'] + [f'day_{i}' for i in range(1, days_in_2025 + 1)]

print(f'\n✓ Final dataframe structure:')
print(f'  - Shape: {df_wide.shape[0]:,} rows × {df_wide.shape[1]} columns')
print(f'  - Columns: {list(df_wide.columns[:5])} ... {list(df_wide.columns[-3:])}')
print(f'\nFirst 3 rows, first 8 columns:')
print(df_wide.iloc[:3, :8])


In [ ]:
# Display data quality statistics
print('DATA QUALITY STATISTICS')
print('='*80)

# Calculate completeness per station
day_cols = [col for col in df_wide.columns if col.startswith('day_')]
df_wide['completeness'] = df_wide[day_cols].notna().sum(axis=1) / len(day_cols) * 100

print(f'\nTotal stations: {len(df_wide):,}')
print(f'\nCompleteness statistics (% of days with data):')
print(f'  - Mean: {df_wide["completeness"].mean():.1f}%')
print(f'  - Median: {df_wide["completeness"].median():.1f}%')
print(f'  - Min: {df_wide["completeness"].min():.1f}%')
print(f'  - Max: {df_wide["completeness"].max():.1f}%')

# Show distribution of completeness
print(f'\nCompleteness distribution:')
print(f'  - 100% complete: {(df_wide["completeness"] == 100).sum():,} stations')
print(f'  - >= 90% complete: {(df_wide["completeness"] >= 90).sum():,} stations')
print(f'  - >= 50% complete: {(df_wide["completeness"] >= 50).sum():,} stations')
print(f'  - < 50% complete: {(df_wide["completeness"] < 50).sum():,} stations')

# Remove the completeness column from final output
df_final = df_wide.drop('completeness', axis=1)

print(f'\n✓ Final dataframe ready: {df_final.shape[0]:,} rows × {df_final.shape[1]} columns')


In [ ]:
# Display sample of the final dataframe
print('SAMPLE OF FINAL DATAFRAME')
print('='*80)
print('\nFirst 5 stations, first 10 columns:')
print(df_final.iloc[:5, :10])

print('\nFirst 5 stations, last 10 columns:')
print(df_final.iloc[:5, -10:])

# Show a station with good coverage
good_station_idx = df_wide.sort_values('completeness', ascending=False).index[0]
print(f'\nExample of well-covered station: {df_wide.loc[good_station_idx, "station_id"]}')
print(f'Completeness: {df_wide.loc[good_station_idx, "completeness"]:.1f}%')
print('First 15 days of data:')
print(df_final.loc[good_station_idx, ['station_id', 'year'] + [f'day_{i}' for i in range(1, 16)]])


In [ ]:
# Save to Parquet and CSV
import os

output_file = 'tobs_2025_wide.parquet'

print(f'Saving dataframe to {output_file}...')
df_final.to_parquet(output_file, index=False, compression='snappy')
print(f'✓ Saved to {output_file}')

# Also save a CSV sample (first 1000 rows) for easy viewing
csv_sample = 'tobs_2025_wide_sample.csv'
df_final.head(1000).to_csv(csv_sample, index=False)
print(f'✓ Saved sample (first 1000 rows) to {csv_sample}')

print(f'\nFile sizes:')
print(f'  - {output_file}: {os.path.getsize(output_file) / (1024*1024):.2f} MB')
print(f'  - {csv_sample}: {os.path.getsize(csv_sample) / (1024*1024):.2f} MB')


## Summary

The final dataframe `df_final` contains:
- **Columns**: `station_id`, `year`, `day_1`, `day_2`, ..., `day_365`
- **Rows**: One per station with TOBS data in 2025
- **Values**: TOBS temperature observations (in tenths of degrees C)
- **Missing data**: Represented as NaN

### Notes:
- TOBS values are in tenths of degrees Celsius (divide by 10 to get °C)
- 2025 is not a leap year, so it has 365 days
- Some stations may have incomplete data coverage
- The dataframe is saved as both Parquet (full data) and CSV (sample)
